In [1]:
#!pip3 install pandas

In [4]:
#from pyspark.sql import SQLContext
from py4j.java_gateway import java_import
#import pandas as pd

In [16]:
spark = SparkSession \
    .builder \
    .appName('restlogs_qlik') \
    .enableHiveSupport()\
    .config('spark.sql.hive.thriftServer.singleSession', True)\
    .config('hive.server2.authentication','NONE')\
    .config('hive.server2.thrift.port', '10003')\
    .config('hive.server2.singleSession',"True")\
    .config('hive.server2.thrift.singleSession',"True")\
    .getOrCreate()

In [17]:
sc=spark.sparkContext
sc.setLogLevel('INFO')

In [18]:
java_import(sc._gateway.jvm, "")
sc._gateway.jvm.org.apache.spark.sql.hive.thriftserver.HiveThriftServer2.startWithContext(spark._jwrapped)

JavaObject id=o178

In [19]:
muvyUri = "https://imdb.hriks.com/person/"

In [20]:
inputDf = spark.read.option('header', 'true').csv('./testinp.csv')
inputDf.createOrReplaceTempView('testinptbl')

In [21]:
spark.sql("select * from testinptbl").show()

+------+--------+-----+
|format|category|limit|
+------+--------+-----+
|  json|  actors|  100|
+------+--------+-----+



In [22]:
prmstest = { 'url' : muvyUri, 'input': "testinptbl", 'method' : 'GET', 'readTimeout' : '10000', 'connectionTimeout' : '2000', 'partitions' : '10'}

In [23]:
testDF = spark.read.format("org.apache.dsext.spark.datasource.rest.RestDataSource").options(**prmstest).load()

In [24]:
testDF.printSchema()

root
 |-- category: string (nullable = true)
 |-- format: string (nullable = true)
 |-- limit: string (nullable = true)
 |-- output: struct (nullable = true)
 |    |-- count: long (nullable = true)
 |    |-- next: string (nullable = true)
 |    |-- previous: string (nullable = true)
 |    |-- results: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- categories: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- country: string (nullable = true)
 |    |    |    |-- full_name: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)



In [25]:
from pyspark.sql import types as T
import pyspark.sql.functions as F


def flatten(df, dbg = False):
    complex_fields = dict([
        (field.name, field.dataType) 
        for field in df.schema.fields 
        if isinstance(field.dataType, T.ArrayType) or isinstance(field.dataType, T.StructType)
    ])
    
    if dbg:
        print(complex_fields)
        print(len(complex_fields))
    
    qualify = list(complex_fields.keys())[0] + "_"
    
    if dbg:
        print(qualify)

    while len(complex_fields) != 0:
        col_name = list(complex_fields.keys())[0]
        print(col_name)
        
        if isinstance(complex_fields[col_name], T.StructType):
            expanded = [F.col(col_name + '.' + k).alias(col_name + '_' + k) 
                        for k in [ n.name for n in  complex_fields[col_name]]
                       ]
            print(expanded)
            df = df.select("*", *expanded).drop(col_name)
            df.printSchema()
    
        elif isinstance(complex_fields[col_name], T.ArrayType): 
            df = df.withColumn(col_name, F.explode(col_name))
            df.printSchema()
    
      
        complex_fields = dict([
            (field.name, field.dataType)
            for field in df.schema.fields
            if isinstance(field.dataType, T.ArrayType) or isinstance(field.dataType, T.StructType)
        ])
        
        
    for df_col_name in df.columns:
        df = df.withColumnRenamed(df_col_name, df_col_name.replace(qualify, ""))

    return df

In [26]:
fin_df = flatten(testDF)

output
[Column<b'output.count AS `output_count`'>, Column<b'output.next AS `output_next`'>, Column<b'output.previous AS `output_previous`'>, Column<b'output.results AS `output_results`'>]
root
 |-- category: string (nullable = true)
 |-- format: string (nullable = true)
 |-- limit: string (nullable = true)
 |-- output_count: long (nullable = true)
 |-- output_next: string (nullable = true)
 |-- output_previous: string (nullable = true)
 |-- output_results: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- categories: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- full_name: string (nullable = true)
 |    |    |-- id: long (nullable = true)

output_results
root
 |-- category: string (nullable = true)
 |-- format: string (nullable = true)
 |-- limit: string (nullable = true)
 |-- output_count: long (nullable = true)
 |-- output_next: string (nullable = t

In [27]:
fin_df.printSchema()

root
 |-- category: string (nullable = true)
 |-- format: string (nullable = true)
 |-- limit: string (nullable = true)
 |-- count: long (nullable = true)
 |-- next: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- results_categories: string (nullable = true)
 |-- results_country: string (nullable = true)
 |-- results_full_name: string (nullable = true)
 |-- results_id: long (nullable = true)



In [28]:
fin_df.createOrReplaceTempView("MuvyTbl")

In [15]:
fin_df.registerGlobalView("MuvyTbl2")

AttributeError: 'DataFrame' object has no attribute 'registerGlobalView'

In [16]:
fin_df.createGlobalTempView("MuvyTbl2")

In [17]:
spark.sql("select * from MuvyTbl").show()

+--------+------+-----+-----+--------------------+--------+------------------+---------------+-----------------+----------+
|category|format|limit|count|                next|previous|results_categories|results_country|results_full_name|results_id|
+--------+------+-----+-----+--------------------+--------+------------------+---------------+-----------------+----------+
|  actors|  json|  100|10265|http://imdb.hriks...|    null|             actor|           null| Amit Kumar Gupta|         1|
|  actors|  json|  100|10265|http://imdb.hriks...|    null|           romance|            IND|    hritik Roshan|         2|
|  actors|  json|  100|10265|http://imdb.hriks...|    null|            horror|            IND|    hritik Roshan|         2|
|  actors|  json|  100|10265|http://imdb.hriks...|    null|             actor|           null|      Robert D jr|         3|
|  actors|  json|  100|10265|http://imdb.hriks...|    null|           romance|            IND|    hritik Roshan|         4|
|  actor